In [3]:
import pandas as pd
# Excel 파일의 모든 시트 이름 확인

# 세부 수량 및 수종자료 엑셀 파일 읽기
file_path = r'2024_도로공사_고속도로 정밀도로지도 데이터명세서_250530_1018.xlsx'

xl = pd.ExcelFile(file_path)
print("사용 가능한 시트 목록:")
print(xl.sheet_names)
sheet_names = xl.sheet_names
sheet_names.remove('총괄')
print(sheet_names)

사용 가능한 시트 목록:
['총괄', '001_A1', '002_A2', '003~009_A3', '010~014_A4', '015_A5', '016_AE', '017_B1', '018_B2', '019_B3', '020_BE', '021_BE', '022~023_C2', '024~025_C3', '026_C3', '027_C4', '028_C5', '029_CE', '030_DE', '031~032_DE', '033~036_DE', '037_DE', '038_DE', '039_DE', '040_DE', '041_DE', '042_DE', '043_DE', '044_DE', '045_DE', '046_EC01', '047_ER16', '048_EC08', '049_ER17', '050_ER20', '051_ER21', '052_ER30', '053_ER31', '054_ER32', '055_ER36', '056_ER40', '057_ER41', '058_ER42', '059_ER43', '060_ER44', '061_ER45', '062_ER46', '063_ER47', '064_EC09', '065_ER49']
['001_A1', '002_A2', '003~009_A3', '010~014_A4', '015_A5', '016_AE', '017_B1', '018_B2', '019_B3', '020_BE', '021_BE', '022~023_C2', '024~025_C3', '026_C3', '027_C4', '028_C5', '029_CE', '030_DE', '031~032_DE', '033~036_DE', '037_DE', '038_DE', '039_DE', '040_DE', '041_DE', '042_DE', '043_DE', '044_DE', '045_DE', '046_EC01', '047_ER16', '048_EC08', '049_ER17', '050_ER20', '051_ER21', '052_ER30', '053_ER31', '054_ER32', '0

In [4]:
import pandas as pd
import re
# 세부 수량 및 수종자료 엑셀 파일 읽기
file_path = r'2024_도로공사_고속도로 정밀도로지도 데이터명세서_250530_1018.xlsx'

def get_number_range(sheet_name):
    """시트명에서 숫자 범위를 추출하는 함수"""
    if '~' in sheet_name:
        # 범위가 있는 경우 (예: '003~009_A3')
        numbers = sheet_name.split('_')[0]
        start, end = numbers.split('~')
        start_num = int(start.lstrip('0'))
        end_num = int(end.lstrip('0'))
        return list(range(start_num, end_num + 1))
    else:
        # 단일 숫자인 경우
        return [int(sheet_name.split('_')[0].lstrip('0'))]

# 각 시트 처리
for sheet_name in sheet_names:
    # 레이어명 가져오기 (J열 2행)
    layer_name_df = pd.read_excel(
        file_path,
        sheet_name=sheet_name,
        usecols='J',
        nrows=2,
        header=None
    )
    layer_name = layer_name_df.iloc[1, 0]  # 2번째 행(인덱스 1)의 값
    
    # 먼저 전체 데이터를 읽어서 첫 번째 빈 행을 찾기
    temp_df = pd.read_excel(
        file_path, 
        sheet_name=sheet_name,
        header=4,
        usecols='A:L'  # A부터 L까지의 열만 읽기
    )

    # 첫 번째 모든 열이 NaN인 행 찾기
    first_empty_row = None
    for idx, row in temp_df.iterrows():
        if row.isna().all():
            first_empty_row = idx
            break

    # 실제 데이터 읽기 (첫 번째 표만)
    df = pd.read_excel(
        file_path, 
        sheet_name=sheet_name,
        header=4,
        nrows=first_empty_row if first_empty_row else None,
        usecols='A:L'  # A부터 L까지의 열만 읽기
    )
    
    # NaN인 열 제거
    df.dropna(axis=1, how='all', inplace=True)
    
    # 레이어명 칼럼 추가
    df['레이어명'] = layer_name
    
    # 시트명에서 숫자 범위를 가져와서 해당하는 모든 데이터프레임 생성
    number_range = get_number_range(sheet_name)
    for num in number_range:
        globals()[f'df_{num}'] = df.copy()
    
print("데이터프레임이 생성되었습니다.")

데이터프레임이 생성되었습니다.


In [5]:
df_65

,속성명,속성,KEY,데이터타입,제약조건,레이어명
0,ID,ID,PK,VARCHAR2(14),UNIQUE/NOT NULL,ER49_A9990013
1,MNGM,관리지사,-,VARCHAR2(30),NOT NULL,ER49_A9990013
2,DATE,업데이트일자,-,VARCHAR2(8),NOT NULL,ER49_A9990013
3,REMARK,비고,-,VARCHAR2(30),-,ER49_A9990013
4,KMPOST,이정,-,"FLOAT(10,4)",-,ER49_A9990013
5,SYSTEM_ID,도로대장\n시설물ID,FK,VARCHAR2(40),-,ER49_A9990013
6,SRC_ID,참조데이터ID,-,VARCHAR2(40),-,ER49_A9990013
7,SRC_VER,참조데이터 버전,-,VARCHAR2(40),-,ER49_A9990013
8,SRC_TYPE,참조데이터 유형,-,VARCHAR2(40),-,ER49_A9990013


In [80]:
# 생성된 데이터프레임 목록 확인
all_numbers = []
for sheet_name in sheet_names:
    all_numbers.extend(get_number_range(sheet_name))
all_numbers = sorted(set(all_numbers))  # 중복 제거 및 정렬

print("생성된 데이터프레임 목록:")
for num in all_numbers:
    df_name = f'df_{num}'
    if df_name in globals():
        df = globals()[df_name]
        layer_name = df['레이어명'].iloc[0]  # 첫 번째 행의 레이어명 가져오기
        print(f"- {df_name}: {len(df)} 행, {len(df.columns)} 열, 레이어명: {layer_name}")

print("\n고유한 레이어명 목록:")
unique_layers = set()
for num in all_numbers:
    df_name = f'df_{num}'
    if df_name in globals():
        df = globals()[df_name]
        unique_layers.add(df['레이어명'].iloc[0])
print('\n'.join(sorted(unique_layers)))

생성된 데이터프레임 목록:
- df_1: 11 행, 6 열, 레이어명: A1_NODE
- df_2: 18 행, 6 열, 레이어명: A2_LINK
- df_3: 12 행, 6 열, 레이어명: A3_DRIVEWAYSECTION
- df_4: 12 행, 6 열, 레이어명: A3_DRIVEWAYSECTION
- df_5: 12 행, 6 열, 레이어명: A3_DRIVEWAYSECTION
- df_6: 12 행, 6 열, 레이어명: A3_DRIVEWAYSECTION
- df_7: 12 행, 6 열, 레이어명: A3_DRIVEWAYSECTION
- df_8: 12 행, 6 열, 레이어명: A3_DRIVEWAYSECTION
- df_9: 12 행, 6 열, 레이어명: A3_DRIVEWAYSECTION
- df_10: 13 행, 6 열, 레이어명: A4_SUBSIDIARYSECTION
- df_11: 13 행, 6 열, 레이어명: A4_SUBSIDIARYSECTION
- df_12: 13 행, 6 열, 레이어명: A4_SUBSIDIARYSECTION
- df_13: 13 행, 6 열, 레이어명: A4_SUBSIDIARYSECTION
- df_14: 13 행, 6 열, 레이어명: A4_SUBSIDIARYSECTION
- df_15: 13 행, 6 열, 레이어명: A5_PARKINGLOT
- df_16: 12 행, 6 열, 레이어명: AE_SLOPE
- df_17: 11 행, 6 열, 레이어명: B1_SAFETYSIGN
- df_18: 12 행, 6 열, 레이어명: B2_SURFACELINEMARK
- df_19: 12 행, 6 열, 레이어명: B3_SURFACEMARK
- df_20: 11 행, 6 열, 레이어명: BE_ROADSIGN
- df_21: 10 행, 6 열, 레이어명: BE_VMS
- df_22: 11 행, 6 열, 레이어명: C2_KILOPOST
- df_23: 11 행, 6 열, 레이어명: C2_KILOPOST
- df_24: 13 행, 6 열, 레이어명: C3